# Optuna study

> Combine it with papermill and wandb for seamless hyperparameter tuning

In [1]:
import os
import optuna
from tsai.optuna import *
import papermill as pm
from tsai.optuna import run_optuna_study
from fastcore.basics import *
from optuna.distributions import *
from optuna.samplers import TPESampler
import wandb

/usr/local/pip-global/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = AttrDict(
    study_name = 'general_study_extended', # name of the Optuna study
    study_type = 'bayesian', # 'bayesian' or 'gridsearch' or 'random'
    n_trials = 50, # number of trials
    train_nb = f'{os.getcwd()}/solfsmy_train.ipynb', # path to the notebook to be executed
    search_space = {
        "arch.attn_dropout": DiscreteUniformDistribution(0.0, 0.5, 0.1),
        "arch.d_model": IntUniformDistribution(32, 512, 32),
        "arch.d_ff": IntUniformDistribution(32, 512, 32),
        "arch.decomposition": CategoricalDistribution([True, False]),
        "arch.dropout": DiscreteUniformDistribution(0.0, 0.5, 0.1), 
        "arch.individual": CategoricalDistribution([True, False]),
        "arch.n_layers": IntUniformDistribution(1, 6, 1),
        "arch.n_heads": CategoricalDistribution([2, 4, 8, 16, 32]),
        "init_weights": CategoricalDistribution([True, False]), # true = kaiming
        "lookback": CategoricalDistribution([18, 24, 36, 128, 192])  # Change here
    },
    # Add extra parameters that are fixed, but not part of the search space
    extra_params = {
        "n_epoch": 50,
        "bs": 128
    },
    use_wandb = True,
    wandb_mode = 'offline'
)

In [3]:
def create_objective(train_nb, search_space, extra_params=None, use_wandb=False):
    """
        Create objective function to be minimized by Optuna.
        Inputs:
            trial: Optuna trial object
            train_nb: path to the training notebook
            search_vars: keys of the search space to be used
            wandb_group: name of the wandb group to be used
        Output:
            valid_loss: validation loss
    """
    def objective(trial:optuna.Trial):
        # Define the parameters to be passed to the training notebook through papermill
        pm_parameters = {}
        for k,v in search_space.items():
            pm_parameters['config.' + k] = trial._suggest(k, v)

        # Add the extra parameters to the dictionary. The key of every parameter 
        # must be 'config.<param_name>'
        if extra_params is not None:
            for k,v in extra_params.items():
                pm_parameters['config.' + k] = v
                
        # If using wandb, enable that in the training runs, all of them gathered
        # into a group (NOTE: The train nb must have and use these config arguments)
        if use_wandb:
            pm_parameters['config.use_wandb'] = True
            pm_parameters['config.wandb_group'] = config.study_name + '_runs'

        # Call the training notebook using papermill (don't print the output)
        stdout_file = open('tmp/pm_stdout.txt', 'w')
        stderr_file = open('tmp/pm_stderr.txt', 'w')

        pm.execute_notebook(
            train_nb,
            './tmp/pm_output.ipynb',
            parameters = pm_parameters,
            stdout_file = stdout_file,
            stderr_file = stderr_file
        )

        # Close the output files
        stdout_file.close()
        stderr_file.close()

        # Get the output value of interest from the source notebook
        %store -r valid_loss
        return valid_loss

    return objective

In [4]:
obj = create_objective(config.train_nb, config.search_space, 
                       extra_params=config.extra_params, use_wandb=True)
study = run_optuna_study(obj, study_type='bayesian', direction='minimize', path='./tmp',
                 study_name=config.study_name, n_trials=config.n_trials)

[I 2024-07-05 11:09:17,373] A new study created in memory with name: general_study_extended
  0%|          | 0/50 [00:00<?, ?it/s]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group
Input notebook does not contain a cell with tag 'parameters'
  0%|          | 0/50 [00:01<?, ?it/s]


[W 2024-07-05 11:09:19,022] Trial 0 failed with parameters: {'arch.attn_dropout': 0.30000000000000004, 'arch.d_model': 160, 'arch.d_ff': 96, 'arch.decomposition': True, 'arch.dropout': 0.0, 'arch.individual': True, 'arch.n_layers': 2, 'arch.n_heads': 2, 'init_weights': False, 'lookback': 18} because of the following error: PapermillExecutionError(0, 1, '# Parameters\nconfig.arch.attn_dropout = 0.30000000000000004\nconfig.arch.d_model = 160\nconfig.arch.d_ff = 96\nconfig.arch.decomposition = True\nconfig.arch.dropout = 0.0\nconfig.arch.individual = True\nconfig.arch.n_layers = 2\nconfig.arch.n_heads = 2\nconfig.init_weights = False\nconfig.lookback = 18\nconfig.n_epoch = 50\nconfig.bs = 128\nconfig.use_wandb = True\nconfig.wandb_group = "general_study_extended_runs"\n', 'NameError', "name 'config' is not defined", ['\x1b---------------------------------------------------------------------------\x1b', '\x1bNameError\x1b                                 Traceback (most recent call last)', 

PapermillExecutionError: 
---------------------------------------------------------------------------
Exception encountered at "In [1]":
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[1], line 2
      1 # Parameters
----> 2 config.arch.attn_dropout = 0.30000000000000004
      3 config.arch.d_model = 160
      4 config.arch.d_ff = 96

NameError: name 'config' is not defined


In [ ]:
run = wandb.init(config=config, mode=config.wandb_mode, 
                 job_type='optuna-study') if config.use_wandb else None

In [ ]:
if run is not None:
    run.log(dict(study.best_params, **{'best_value': study.best_value, 
                                       'best_trial_number': study.best_trial.number}))
    run.log_artifact(f'./tmp/{config.study_name}.pkl', type='optuna_study')
    run.log({
        'contour': optuna.visualization.plot_contour(study),
        'edf': optuna.visualization.plot_edf(study),
        'intermediate_values': optuna.visualization.plot_intermediate_values(study),
        'optimization_history': optuna.visualization.plot_optimization_history(study),
        'parallel_coordinate' : optuna.visualization.plot_parallel_coordinate(study),
        'param_importances': optuna.visualization.plot_param_importances(study),
        'slice': optuna.visualization.plot_slice(study)
    })

In [ ]:
if run is not None:
    run.finish()